# AOC2019 - Day 13

Today's challenge: [Day 13](https://adventofcode.com/2019/day/13)

Data are manually downloaded into `./data/input_13.txt`

In [20]:
with open('./data/input_13.txt') as f:
    raw = f.read()

In [25]:
class Intcode:
    def __init__(self, xs, pos=0, i=None):
        self.xs = xs
        self.pos = pos
        self.rel_base = 0
        self.i = i
        self.res = 0
        
    def display(self):
        return dict(enumerate(self.xs))
    
    @staticmethod
    def parse_op(x):
        params, op = divmod(x, 100)
        params, p1 = divmod(params, 10)
        params, p2 = divmod(params, 10)
        p3 = params % 10
        return op, p1, p2, p3
    
    def get_value(self,i,p):
        if p == 0:
            return self.xs[i]   
        elif p == 1:
            return i
        elif p == 2:
            i_ = i + self.rel_base
            return self.xs[i_]
        else:
            print("WTF:", p, i)
    
    def set_value(self,i,p):
        if p == 0:
            return i
        elif p == 2:
            i_ = i + self.rel_base
            return i_
        else:
            print("WTF:", p, i)
    
    def op1(self,i1,i2,out,ps):
        p1, p2, pout = ps
        i1_ = self.get_value(i1, p1)
        i2_ = self.get_value(i2, p2)
        out_ = self.set_value(out, pout)
        self.xs[out_] = i1_ + i2_
        return 4
    
    def op2(self,i1,i2,out,ps):
        p1, p2, pout = ps
        i1_ = self.get_value(i1, p1)
        i2_ = self.get_value(i2, p2)
        out_ = self.set_value(out, pout)
        self.xs[out_] = i1_ * i2_
        return 4

    def op3(self,x,i,ps):
        p, *_ = ps
        i_ = self.set_value(i, p)
        self.xs[i_] = x
        return 2

    def op4(self,i,ps):
        p, *_ = ps
        i_ = self.get_value(i, p)
        self.res = i_
        if i_ == -1:
            pass
            #print("TEMP: ", [self.xs[x] for x in [2391,2392,2394]])
        return 2
    
    def op5(self,i,out,ps):
        pi, pout, _ = ps
        i_ = self.get_value(i, pi)
        #out_ = self.set_value(out, pout)
        out_ = self.get_value(out, pout)
        if i_ != 0:
            self.pos = out_
            return 0
        return 3
    
    def op6(self,i,out,ps):
        pi, pout, _ = ps
        i_ = self.get_value(i, pi)
        out_ = self.get_value(out, pout)
        if i_ == 0:
            self.pos = out_
            return 0
        return 3
    
    def op7(self,i1,i2,out,ps):
        p1, p2, pout = ps
        i1_ = self.get_value(i1, p1)
        i2_ = self.get_value(i2, p2)
        out_ = self.set_value(out, pout)
        self.xs[out_] = 1 if i1_ < i2_ else 0
        return 4
    
    def op8(self,i1,i2,out,ps):
        p1, p2, pout = ps
        i1_ = self.get_value(i1, p1)
        i2_ = self.get_value(i2, p2)
        out_ = self.set_value(out, pout)
        self.xs[out_] = 1 if i1_ == i2_ else 0
        return 4
    
    def op9(self,i,ps):
        p, *_ = ps
        i_ = self.get_value(i, p)
        self.rel_base += i_
        return 2
    
    def op99(self):
        return 0

    def run(self, x=0):
        while True:
            op, *params = self.parse_op(self.xs[self.pos])
            if self.pos == 433:
                pass
                #print(op, self.pos, self.rel_base, self.xs[2394])
            if op%100 == 1:
                i1, i2, out = [self.xs[x] for x in range(self.pos+1,self.pos+4)]
                self.pos += self.op1(i1, i2, out, params)
            elif op%100 == 2:
                i1, i2, out = [self.xs[x] for x in range(self.pos+1,self.pos+4)]
                self.pos += self.op2(i1, i2, out, params)
            elif op%100 == 3:
                i = self.xs[self.pos+1]
                self.pos += self.op3(x, i, params)
            elif op%100 == 4:
                i = self.xs[self.pos+1]
                self.pos += self.op4(i, params)
                return self.res, True
            elif op%100 == 5:
                i, out = [self.xs[x] for x in range(self.pos+1,self.pos+3)]
                p = self.op5(i, out, params)
                self.pos += p
            elif op%100 == 6:
                i, out = [self.xs[x] for x in range(self.pos+1,self.pos+3)]
                p = self.op6(i, out, params)
                self.pos += p
            elif op%100 == 7:
                i1, i2, out = [self.xs[x] for x in range(self.pos+1,self.pos+4)]
                self.pos += self.op7(i1, i2, out, params)
            elif op%100 == 8:
                i1, i2, out = [self.xs[x] for x in range(self.pos+1,self.pos+4)]
                self.pos += self.op8(i1, i2, out, params)
            elif op%100 == 9:
                i1 = self.xs[self.pos+1]
                self.pos += self.op9(i1, params)
            elif op%100 == 99:
                self.op99()
                return self.res, False
            else:
                print("invalid instruction!", op)
                break

In [133]:
class Robot:
    def __init__(self, brain):
        self.p = brain
        self.panel = defaultdict(int)
        
    def __repr__(self):
        xs, ys = zip(*self.panel.keys())
        x0, x1 = min(xs), max(xs)
        y0, y1 = min(ys), max(ys)
        res = []
        for y in range(y0, y1+1):
            row = ""
            for x in range(x0,x1+1):
                if self.panel[(x,y)] == 0:
                    row += ' '
                elif self.panel[(x,y)] == 1:
                    row += '|'
                elif self.panel[(x,y)] == 2:
                    row += '#'
                elif self.panel[(x,y)] == 3:
                    row += '-'
                elif self.panel[(x,y)] == 4:
                    row += 'O'
                else:
                    print("WTF")
            res.append(row)
        return "\n".join(res)
                            
    def add_symbol(self, x, y, v):
        self.panel[(x,y)] = v
        
    def run(self, xs=[]):
        c = True
        step = 0
        extra_step = 0
        j_pos = 0
        idx = 0
        while c:
            out1, _ = self.p.run(j_pos)
            out2, _ = self.p.run()
            out3, c = self.p.run()
            if out1 != -1:
                self.add_symbol(out1, out2, out3)
            step += 1
            if step > 875 and step % 2 == 0:
                #print(self)
                if out1 == -1 and out2 == 0:
                    pass
                    #print("Score: ", out3)
                if idx < len(xs):
                    j_pos = xs[idx]
                    idx += 1
                else:
                    print(self)
                    print(extra_step+1)
                    extra_step += 1
                    i = None
                    while not i in ['0','-','+','','c']:
                        i = input()
                    if i == 'c':
                        return
                    elif i == '-':
                        i = -1
                    elif i == '':
                        i = 0
                    elif i == '+':
                        i = 1
                    j_pos = int(i)

In [134]:
with open('./data/input_13.txt') as f:
    raw = f.read()

In [257]:
from collections import defaultdict, Counter
data = defaultdict(int)
data.update({(i,int(x)) for i,x in enumerate(raw.strip().split(","))})
data[0] = 2
p = Intcode(data)
r = Robot(p)
r.run(
    10*[0]+29*[-1]+6*[1]+26*[0]+4*[-1]+16*[0]+14*[1]+4*[-1]+80*[0]+8*[1]+1240*[0]+20*[1]+
    3*[0]+12*[-1]+8*[0]+16*[1]+57*[0]+10*[1]+20*[0]+2*[-1]+2*[0]+10*[1]+17*[0]+18*[-1]+13*[0]+
    5*[1]+30*[0]+4*[-1]+27*[0]+12*[1]+50*[0]+8*[-1]+75*[0]+3*[-1]+440*[0]+15*[1]+6*[0]+43*[-1]+
    22*[0]+17*[1]+25*[0]+20*[-1]+30*[0]+39*[1]+2*[-1]+62*[0]+5*[-1]+46*[0]+29*[-1]+2*[1]+
    78*[0]+4*[1]+128*[0]+4*[-1]+4*[0]+36*[1]+20*[0]+105*[0]+18*[-1]+57*[0]+28*[1]+44*[-1]+
    60*[0]+5*[-1]+300*[0]+31*[1]+97*[0]+4*[1]+64*[0]+50*[-1]+2*[1]+33*[0]+37*[1]+43*[0]+6*[1]+
    2*[0]+6*[-1]+38*[0]+36*[-1]+25*[0]+2*[1]+24*[0]+
    42*[1]+
    21*[0]+
    2*[1]+
    10*[0]+
    43*[-1]+
    22*[0]+
    41*[1]+
    22*[0]+
    29*[-1]+
    43*[1]+
    21*[0]+
    44*[-1]+
    21*[0]+
    2*[-1]+
    20*[0]+
    91*[0]
)

|||||||||||||||||||||||||||||||||||
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                #                |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|          O                      |
|         -                       |
|                                 |
1

|||||||||||||||||||||||||||||||||||
|                                 |
|                        


|||||||||||||||||||||||||||||||||||
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                    O            |
|                                 |
|                #                |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|         -                       |
|                                 |
11

|||||||||||||||||||||||||||||||||||
|                                 |
|                      


|||||||||||||||||||||||||||||||||||
|                                 |
|                              O  |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                #                |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|         -                       |
|                                 |
21

|||||||||||||||||||||||||||||||||||
|                               O |
|                      


|||||||||||||||||||||||||||||||||||
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                        O        |
|                                 |
|                                 |
|                                 |
|                #                |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|         -                       |
|                                 |
31

|||||||||||||||||||||||||||||||||||
|                                 |
|                      

+
|||||||||||||||||||||||||||||||||||
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                #                |
|                                 |
|                                 |
|                                 |
|                                 |
|               O                 |
|                                 |
|                                 |
|                                 |
|          -                      |
|                                 |
41
+
|||||||||||||||||||||||||||||||||||
|                                 |
|                    

...

-
|||||||||||||||||||||||||||||||||||
|                                 |
|                                 |
|                                 |
|                O                |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                #                |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|             -                   |
|                                 |
141
-
|||||||||||||||||||||||||||||||||||
|                                 |
|                   

-
|||||||||||||||||||||||||||||||||||
|                                 |
|                                 |
|           O                     |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                #                |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|        -                        |
|                                 |
151
-
|||||||||||||||||||||||||||||||||||
|                                 |
|                   


|||||||||||||||||||||||||||||||||||
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|   O                             |
|                                 |
|                                 |
|                #                |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|      -                          |
|                                 |
161

|||||||||||||||||||||||||||||||||||
|                                 |
|                     

+
|||||||||||||||||||||||||||||||||||
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                #                |
|                                 |
|                                 |
|                                 |
|                                 |
|     O                           |
|                                 |
|                                 |
|                                 |
|        -                        |
|                                 |
171
+
|||||||||||||||||||||||||||||||||||
|                                 |
|                   

+
|||||||||||||||||||||||||||||||||||
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                                 |
|                #                |
|                                 |
|                                 |
|                                 |
|            O                    |
|                                 |
|                                 |
|                                 |
|                                 |
|           -                     |
|                                 |
181
+
|||||||||||||||||||||||||||||||||||
|                                 |
|                   

In [258]:
r.p.xs[386]

15410